In [1]:
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

In [2]:
# Define data directories and parameters
train_dir = "D:/SALMAN/Banpu/Banpu_5/train"
val_dir = "D:/SALMAN/Banpu/Banpu_5/valid"
test_dir = "D:/SALMAN/Banpu/Banpu_5/test"
img_size = (224, 224)  # Adjust to your model's input size
batch_size = 32

In [3]:
# Create a new ImageDataGenerator for data augmentation
datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [4]:
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

Found 3531 images belonging to 2 classes.


In [5]:
val_generator = datagen.flow_from_directory(
    val_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

Found 756 images belonging to 2 classes.


In [6]:
# Define a pre-trained model
base_model = MobileNetV2(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=predictions)

In [7]:
# Compile the model
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [8]:
# Train the model
model.fit(train_generator,
          epochs=20,  # Increase the number of epochs
          steps_per_epoch=len(train_generator),
          validation_data=val_generator,
          validation_steps=len(val_generator))

Epoch 1/20
111/111 [==============================] - 1011s 9s/step - loss: 0.3616 - accuracy: 0.8236 - val_loss: 0.9060 - val_accuracy: 0.6799
Epoch 2/20
111/111 [==============================] - 1124s 10s/step - loss: 0.1856 - accuracy: 0.9224 - val_loss: 0.7699 - val_accuracy: 0.7368
Epoch 3/20
111/111 [==============================] - 1202s 11s/step - loss: 0.1434 - accuracy: 0.9402 - val_loss: 1.0215 - val_accuracy: 0.7460
Epoch 4/20
111/111 [==============================] - 1131s 10s/step - loss: 0.1165 - accuracy: 0.9499 - val_loss: 0.5448 - val_accuracy: 0.8056
Epoch 5/20
111/111 [==============================] - 1103s 10s/step - loss: 0.1036 - accuracy: 0.9584 - val_loss: 0.6027 - val_accuracy: 0.8386
Epoch 6/20
111/111 [==============================] - 720s 6s/step - loss: 0.0905 - accuracy: 0.9640 - val_loss: 0.4351 - val_accuracy: 0.8598
Epoch 7/20
111/111 [==============================] - 598s 5s/step - loss: 0.0806 - accuracy: 0.9666 - val_loss: 0.1805 - val_accurac

In [9]:
# Evaluate the model on the test set
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

In [10]:
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

Found 757 images belonging to 2 classes.


In [11]:
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

24/24 [==============================] - 21s 856ms/step - loss: 0.0287 - accuracy: 0.9921
Test Loss: 0.0287482850253582, Test Accuracy: 0.9920739531517029


In [12]:
model.save("test_7.h5")

c:\Users\Salman\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [2]:
from tensorflow.keras.preprocessing import image
import numpy as np
from tensorflow.keras.models import load_model

img_size = (224, 224)

# Load the single image
# image_path = "C:/Users/salman.nishan/Downloads/Banpu_5/data/original_bad/17.jpg"
image_path = "C:/Users/salman.nishan/Downloads/Banpu_5/data/original_bad/123.jpg"
# image_path = "C:/Users/salman.nishan/Downloads/Banpu_5/data/original_good/86.png"

img = image.load_img(image_path, target_size=img_size)
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0  # Normalize pixel values to the range [0, 1]

loaded_model = load_model("test_7.h5")

# Use the loaded model for prediction
prediction = loaded_model.predict(img_array)
print(prediction)

# Interpret the prediction
predicted_class = "good weld" if prediction > 0.8 else "bad weld"
print("Predicted class:", predicted_class)


1/1 [==============================] - 0s 456ms/step
[[0.30517203]]
Predicted class: bad weld
